In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mplleaflet
import geopandas as geopd
import srtm
import gpxpy
import networkx as nx
import process_gpx_data as gpx_process
import os

In [2]:
# try and load file
gdf = geopd.read_file('./data/Boulder_Area_Trails.geojson') # 'Trails.geojson')
print("Trails: ", gdf.columns)

gdf_small = geopd.read_file('./data/Trails.geojson')
#print("Trails Dissolved: ", gdf.columns)

Trails:  Index(['OBJECTID', 'TRAILNAME', 'ALIAS', 'LENGTH', 'PEDESTRIAN', 'BIKE',
       'HORSE', 'OHV', 'TRAILTYPE', 'SURTYPE', 'STEWARD', 'DATAOWNER',
       'COMMENTS', 'SHAPESTLength', 'DOGS', 'GlobalID', 'BATTrailID',
       'geometry'],
      dtype='object')


In [3]:
trail_names_full = gdf.TRAILNAME

In [4]:
trail_names_short = gdf_small.FEATURE_NAME

In [5]:
for k in trail_names_full:
    if not k is None:
        if 'Mud' in k or 'mud' in k:
            print(k)

Mud Lake Connector Trail
Mud Lake Access Trail
Mud Lake Access Trail
Nederland Elem. to Mud Lake
Mud Lake to Hwy. 72


In [6]:
gdf.loc[gdf.TRAILNAME=='Mud Lake Connector Trail']

,OBJECTID,TRAILNAME,ALIAS,LENGTH,PEDESTRIAN,BIKE,HORSE,OHV,TRAILTYPE,SURTYPE,STEWARD,DATAOWNER,COMMENTS,SHAPESTLength,DOGS,GlobalID,BATTrailID,geometry
90,91,Mud Lake Connector Trail,None,0.064478,Yes,Yes,Yes,No,Trail,SoftSurface,https://www.bouldercounty.org/open-space/parks...,Boulder County Parks and Open Space,segmented,135.380251,Yes,{1C256E4A-6B85-4221-8C16-9C584F5B5C8E},BCPOS-95,"LINESTRING (-105.51363 39.97616, -105.51352 39..."


In [7]:
gdf_small.loc[0]

OBJECTID                                                   499057
FEATURE_TYPE                                                    3
LOCATION                                                 Mud Lake
FEATURE_NAME                             Mud Lake Connector Trail
SURFACE_TYPE                                              Natural
WIDTH                                                           3
SOURCE                                                        gps
SRC_YEAR                                                     2004
STATUS                                                   existing
OWNER                                                      County
MAINTENANCE                                                County
ENFORCEMENT                                                County
PED                                                           Yes
BIKE                                                          Yes
HORSE                                                         Yes
DOG       

The key columns here are:

    1) SLength = trail length in feet
    
    2) VerticalGain = gain in feet (*need to update with elevation data for consistency*)
    
    3) Z_Min, Z_Max, Z_Mean = min, max, average altitude *see above*
    
    4) Min_Slope, Max_Slope, Avg_Slope = percent grade *see above*
    
    5) geometry = list of coordinates (as LINESTRING object)... 
                  use LINESTRING.coords to get an iterable over the
                  lat long coordinates as tuples. For whatever reason,
                  this appears to be stored as (long, lat)?!?
                  
                

In [8]:
# try and add in elevation data

elevation_data = srtm.get_data()


I can generate elevations from this data using the below dataset (returns elevation in meters!)

In [9]:
# print(elevation_data.get_elevation(temp.coords[0][1], temp.coords[0][0]))

In [10]:
temp = gdf['geometry'][55]

In [11]:
gdf.loc[55]

OBJECTID                                                        56
TRAILNAME                                       Picture Rock Trail
ALIAS                                                             
LENGTH                                                         5.2
PEDESTRIAN                                                     Yes
BIKE                                                           Yes
HORSE                                                          Yes
OHV                                                             No
TRAILTYPE                                                    Trail
SURTYPE                                                SoftSurface
STEWARD          https://www.bouldercounty.org/open-space/parks...
DATAOWNER                      Boulder County Parks and Open Space
COMMENTS                                                 dissolved
SHAPESTLength                                              10964.5
DOGS                                                          

In [12]:

def make_gpx_data(pgdf):
    gpx_data = [None]*len(pgdf)

    for i in np.arange(len(pgdf)):
        gpx_segment = gpxpy.gpx.GPXTrackSegment()    
        try:
            gpx_points = [gpxpy.gpx.GPXTrackPoint(x[1],x[0]) for x in pgdf['geometry'][i].coords]

            gpx_segment.points.extend(gpx_points)
            elevation_data.add_elevations(gpx_segment)

            gpx_data[i] = gpx_segment        
        except:

            # multiple sub-segments
            #   need to stitch these into separate segments
            gpx_data[i] = [None]*len(pgdf['geometry'][i])
            #print(len(gdf['geometry'][i]))

            for seg_i in np.arange(len(pgdf['geometry'][i])):

                gpx_segment = gpxpy.gpx.GPXTrackSegment()
                gpx_segment.points.extend([gpxpy.gpx.GPXTrackPoint(x[1],x[0]) for x in pgdf['geometry'][i][seg_i].coords])
                elevation_data.add_elevations(gpx_segment)

                gpx_data[i][seg_i] = gpx_segment
        
    return gpx_data


Looking at the below, it looks like the distance between any two contiguous GPS points is 9.094 meters

In [13]:
 if False:
    bad_segments_loc = [55, 272, 299, 317, 326, 342, 386, 416, 420, 421, 438]

    max_sub_dist = np.zeros(len(bad_segments_loc))
    min_sub_dist = np.zeros(len(bad_segments_loc))

    for i, iloc in enumerate(bad_segments_loc):



        # get end coords for first segment
        end_lat, end_long = gdf['geometry'].loc[iloc][0].coords[-1][1],\
                            gdf['geometry'].loc[iloc][0].coords[-1][0]

        end_elev = elevation_data.get_elevation(end_lat, end_long)

        sub_dist = np.zeros(len(gdf['geometry'].loc[iloc])-1)

        loop_count = len(gdf['geometry'].loc[iloc])
        for seg_i in np.arange(1, loop_count):

            segment = gdf['geometry'].loc[iloc][seg_i]

            # get the first and last end points of next segment
            start_lat, start_long = segment.coords[0][1], segment.coords[0][0]
            start_elev = elevation_data.get_elevation(start_lat, start_long)

            print(i,seg_i, end_lat, end_long, start_lat, start_long)


            # compute distance
            sub_dist[seg_i-1] = gpxpy.geo.distance(end_lat, end_long, end_elev,
                                             start_lat, start_long, start_elev)


            end_lat, end_long, end_elev = start_lat*1., start_long*1., start_elev*1.


        max_sub_dist[i] = np.max(sub_dist)
        min_sub_dist[i] = np.min(sub_dist)

    #
    # If any of these are below 9.04 we have a problem
    #
    print("Maximums:")
    print(max_sub_dist)
    print(np.min(max_sub_dist))
    print(np.max(max_sub_dist))

    print("Minimums:")
    print(min_sub_dist)
    print(np.min(min_sub_dist))
    print(np.max(min_sub_dist))


In [14]:

def make_plot(pgdf, gpx_data, zoom=True):

    fig, ax = plt.subplots()
    fig.set_size_inches(12,12)

    #m = Basemap(projection='lcc', resolution='h',
     #           width=8.0E4, height=8.0E4, 
     #           lat_0=40.015, lon_0=-105.2705)
    #m.etopo( alpha=0.5)


    for i in np.arange(len(pgdf)):
        if gpx_data[i] is None:
            print(i)
            continue

        elif np.size(gpx_data[i]) == 1:
            data_to_plot = [gpx_data[i]]
        else:
            data_to_plot = gpx_data[i]

        for segment in data_to_plot:
            lats = np.array([x.latitude for x in segment.points])
            longs = np.array([x.longitude for x in segment.points])
            elev = np.array([x.elevation for x in segment.points])
            ax.plot(longs,lats,lw=5)

    ax.scatter(-105.2705,40.015, color = 'black', s = 200)
    chataqua = (-105.2795, 39.9972)
    ax.scatter(chataqua[0], chataqua[1], color = 'black', marker = '*', s = 500)


    sep = 0.025
    if zoom:
        ax.set_ylim( chataqua[1] - sep, chataqua[1]+sep)
        ax.set_xlim( chataqua[0] - sep, chataqua[0]+sep)
    fig.show()
    
    return fig

In [15]:
gpx_data = make_gpx_data(gdf)
fig = make_plot(gdf, gpx_data)

<IPython.core.display.Javascript object>

In [16]:
gpx_data_small = make_gpx_data(gdf_small)
fig = make_plot(gdf_small, gpx_data_small, zoom=False)

<IPython.core.display.Javascript object>

In [17]:
gdf.iloc[0]

OBJECTID                                                         1
TRAILNAME                                                AHC Trail
ALIAS                                                             
LENGTH                                                         0.5
PEDESTRIAN                                                     Yes
BIKE                                                            No
HORSE                                                           No
OHV                                                             No
TRAILTYPE                                                    Trail
SURTYPE                                                SoftSurface
STEWARD          https://www.bouldercounty.org/open-space/educa...
DATAOWNER                      Boulder County Parks and Open Space
COMMENTS                                                 dissolved
SHAPESTLength                                              849.297
DOGS                                                          

In [18]:
#%matplotlib widget
import contextily as ctx
gdf_crs = gdf.copy().to_crs(epsg=3857)
ax = gdf_crs.plot()
ax.figure.set_size_inches(16,16)
ctx.add_basemap(ax)
ax.figure.show()

<IPython.core.display.Javascript object>

# Lets try a data cleaning algorithm

In [19]:
outname = './data/boulder_area_trail_processed'

remake_data = True

if os.path.isfile(outname + '.geojson') and (not remake_data):
    # reload from file!!!
    new_df, nodes, edges = gpx_process.load_trail_df(outname)
    G                    = gpx_process.load_graph(outname)

else:
    new_df, G = gpx_process.process_data(gdf_small, outname = outname, threshold_distance=4.0)
    

#
#
#


Computed 980 nodes for 490 edges
Modifying Nodes. Creating 199 new nodes from merging 501 together
New node list length (678) from previous (980)
0 1 479 0
2 3 1 2
4 5 3 4
6 7 5 6
8 9 480 7
10 8 8 480
12 8 481 480
14 12 9 481
16 12 10 481
18 19 11 12
20 21 482 483
22 23 484 13
24 25 14 15
26 27 16 17
28 29 18 19
30 31 20 21
22 33 484 22
34 35 23 24
36 37 25 26
38 39 27 28
40 41 29 30
42 43 31 32
44 45 33 34
46 47 35 36
48 49 37 38
50 51 39 40
52 53 41 42
54 55 43 44
56 57 45 46
58 59 47 48
60 61 49 50
62 63 51 52
64 65 53 54
66 67 55 56
68 69 485 486
70 71 57 487
72 71 58 487
71 75 487 59
76 77 60 488
78 77 61 488
80 81 62 63
82 83 489 64
84 85 65 490
86 87 66 67
88 89 68 491
89 91 491 69
92 93 70 71
94 95 72 73
96 97 492 74
96 99 492 75
100 101 493 76
102 103 77 494
104 103 78 494
106 107 495 79
108 109 80 81
110 111 82 496
111 113 496 497
113 111 497 496
116 113 83 497
118 119 498 84
120 121 85 499
122 121 86 499
121 125 499 500
126 127 501 87
128 129 88 502
130 131 503 89
132 133 90

/home/aemerick/code/segmendation/autotrail/process_gpx_data.py:480: RuntimeWarning: invalid value encountered in true_divide
  grade       = dz / distances * 100.0            # percent grade!


203 204 524 525
205 211 526 121
212 213 527 122
214 215 123 528
215 217 528 124
218 219 125 529
220 221 530 531
222 223 126 532
224 225 533 127
221 227 531 128
228 219 534 529
212 231 527 129
232 233 130 535
234 233 536 535
234 237 536 131
238 234 132 536
240 241 537 133
242 242 538 538
242 245 538 134
246 246 539 539
248 249 540 135
250 251 136 137
252 253 541 138
254 255 139 542
256 257 140 141
258 259 142 143
260 255 144 542
262 263 145 543
263 252 543 541
263 267 543 146
268 269 147 148
270 271 149 150
272 273 151 152
126 275 501 153
276 277 154 155
278 279 156 157
280 281 158 159
282 283 160 161
284 285 544 545
286 284 162 544
288 289 546 163
290 291 164 547
292 293 165 166
294 295 548 549
295 297 549 167
298 299 168 550
299 301 550 551
302 303 169 170
304 305 171 172
306 306 552 552
308 309 553 173
308 285 553 545
312 313 174 175
314 315 176 554
315 317 554 177
315 319 554 178
320 321 179 555
322 323 180 556
324 325 181 557
326 82 558 489
328 329 559 182
106 331 495 183
332 106 5

IndexError: list index out of range

In [ ]:
[(1,2)] + [x for x in gdf_small.iloc[1]['geometry'].coords] + [(3,4)]

In [ ]:
m_ft = 3.28084
i = 2
for k in ['Z_Min','Z_Max','Z_Mean','SLength']:
    print(k, gdf_small.iloc[i][k] / m_ft)
print('------------------------------------------')
for k in ['max_altitude','min_altitude','distance']:
    print(k, new_df.iloc[i][k])

In [ ]:
def plot(graph, node_order=None,
         weight='distance',
         region = None):
    
    #pos = nx.spring_layout(graph, weight=weight, seed = 12345)
    
    
    pos = {ni : np.array([n['long'],n['lat']]) for ni,n in graph.nodes(data=True)}
    
    pc = nx.draw_networkx_nodes(graph, pos, node_size=200)
    ax = pc.figure.axes[0]

    pc.figure.set_size_inches(8,8)

    #edges = [(u,v)]
    #etraveled = [(u,v) for (u,v,d) in graph.edges(data=True) if d['traversed_count'] > 0]
    #enottraveled = [(u,v) for (u,v,d) in graph.edges(data=True) if d['traversed_count'] == 0]

    colors = 'black'
    #if not (node_order is None):
    #    ecolor_int = [node_order.index(e[0]) for e in etraveled]
    #    colors = magma((np.array(ecolor_int)+5) / ((1.0*len(node_order)+5)))
        
    temp = nx.draw_networkx_edges(graph, pos, edgelist=graph.edges, width=10, edge_color=colors)
    #temp = nx.draw_networkx_edges(graph, pos, edgelist=enottraveled, width=2, style='dashed')
    
    #try:
    #    edge_labels = { (u,v) : "%i"%(d['ShapeLen']) for (u,v,d) in graph.edges(data=True)}
    #except:    
    #    edge_labels = { (u,v) : "%i"%(d['SHAPESTLength']) for (u,v,d) in graph.edges(data=True)}
    #temp = nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, label_pos=0.5)
    
    temp = nx.draw_networkx_labels(graph, pos, labels={n:n for n,data in graph.nodes(data=True)}, font_size=17,
                            font_color = 'black')
    chataqua = (-105.2795, 39.9972)
    ax.scatter(chataqua[0], chataqua[1], color = 'black', marker = '*', s = 1000)


    #sep = 0.025
    #if zoom:
    #    ax.set_ylim( chataqua[1] - sep, chataqua[1]+sep)
    #    ax.set_xlim( chataqua[0] - sep, chataqua[0]+sep)
        
    #print(pos)
    return pc,ax

In [ ]:
%matplotlib notebook

pc,ax = plot(G)
#ax.set_xlim()
pc.figure.show()


chataqua = (-105.2795, 39.9972)
sep = 0.025
#ax.set_ylim( chataqua[1] - sep, chataqua[1]+sep)
#ax.set_xlim( chataqua[0] - sep, chataqua[0]+sep)

#plt.show()